In [1]:
import sys
sys.path.append('C:\My_files\Programming\work')
import pandas as pd
import numpy as np
import re
from openpyxl import load_workbook
from processing import YouScanProcessing as ysp
from gspread_pandas import Spread, Client
import gspread_pandas
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [2]:
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('./google_secret.json', scope)
gc = gspread.authorize(credentials)
sht = gc.open('Skoda SL Report')

In [3]:
df = pd.read_excel('2428.xlsx')

# Collecting tags

In [5]:
model_tags = ('40_', '41_', '42_', '43_', 
              '44_', '45_', '50_')
feature_tags = ('01_', '02_', '03_', '04_', 
                '05_', '06_', '07_', '08_', 
                '09_', '10_', '11_','12_', 
                '13_', '14_', '15_', '16_',
                '17_', '18_', '19_', '20_',
                '21_','22_')

# Упоминание других авто

In [5]:
df_count = pd.DataFrame({'40_RAPID': [],
                         '43_SUPERB': [],
                         '42_KODIAQ': [],
                         '41_OCTAVIA': [],
                         '45_KAROQ': []})
for col in df.columns:
        if col.startswith('80_'):
            df_count = df_count.append(df.groupby(col)[['40_RAPID',
                                                        '43_SUPERB',
                                                        '42_KODIAQ',
                                                        '41_OCTAVIA',
                                                        '45_KAROQ']] \
                                                        .count()).astype(int)
df_count.head()

/tmp/ipykernel_45135/108117484.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_count = df_count.append(df.groupby(col)[['40_RAPID',
/tmp/ipykernel_45135/108117484.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_count = df_count.append(df.groupby(col)[['40_RAPID',
/tmp/ipykernel_45135/108117484.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_count = df_count.append(df.groupby(col)[['40_RAPID',
/tmp/ipykernel_45135/108117484.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_count = df_count.append(df.groupby(col)[['40_RAPID',
/tmp/ipykernel_45135/108117484.py:8: FutureWarning: The frame.append method is deprecate

,40_RAPID,43_SUPERB,42_KODIAQ,41_OCTAVIA,45_KAROQ
80_ВАЗ (Lada),161,8,17,98,2
80_Renault,45,4,10,34,6
80_Kia,110,17,33,113,10
80_Hyundai,120,8,30,62,6
80_Ford,13,8,4,39,1


# Все модели по количеству

In [6]:
md = ['40_RAPID','41_OCTAVIA','42_KODIAQ','43_SUPERB','45_KAROQ']

# Рекомендует / Не рекомендует

In [7]:
temp_dict = {}

for m in md:
    rec = df[(df[m] == m)]['35_Рекомендует'].count()
    not_rec = df[(df[m] == m)]['36_Не рекомендует'].count()
    unknown = df[(df[m] == m) & (df['35_Рекомендует'] != '35_Рекомендует') & \
                 ((df['36_Не рекомендует'] != '36_Не рекомендует'))]['36_Не рекомендует'].isna().sum()
    total = sum([rec, not_rec, unknown])    
    temp_dict[re.sub('\d{2}_','',m)] = [rec, unknown, not_rec, total]

recommend = pd.DataFrame(temp_dict)
recommend = recommend.rename(index={0:'Recommend', 1:'Not defined', 
                                    2:'Not recommend', 3:'Total'}) \
                     .transpose().sort_values(by='Total', ascending=False)

recommend = ysp.percent(recommend,
                div_round=3, to_percent=False,
                calc_columns=recommend.columns, 
                percent_columns=[f'{col} %' for col in recommend.columns])
recommend

/home/krtsh/MEGAsync/Programming/work/Old scripts/skoda/processing.py:55: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(data[calc_columns] \


,Recommend,Not defined,Not recommend,Total,Recommend %,Not defined %,Not recommend %
OCTAVIA,89,3584,15,3688,0.024,0.972,0.004
RAPID,55,2229,6,2290,0.024,0.973,0.003
KODIAQ,35,1737,5,1777,0.020,0.977,0.003
SUPERB,32,595,7,634,0.050,0.938,0.011
KAROQ,6,348,1,355,0.017,0.980,0.003
Total types,217,8493,34,8744,0.025,0.971,0.004


# Выбирает / Владелец

In [8]:
temp_dict = {}

for m in md:
    own = df[(df[m] == m)]['31_Владелец'].count()
    pick = df[(df[m] == m)]['32_Выбирает авто'].count()
    unknown = df[(df[m] == m) & (df['31_Владелец'] != '31_Владелец') & \
                 ((df['32_Выбирает авто'] != '32_Выбирает авто'))]['31_Владелец'].isna().sum()
    total = sum([own, pick, unknown])
    temp_dict[re.sub('\d{2}_','',m)] = [own, unknown, pick, total]

owners = pd.DataFrame(temp_dict)
owners = owners.rename(index={0:'Owner', 1:'Not defined', 
                              2:'Potential buyer', 3:'Total'}) \
                .transpose().sort_values(by='Total', ascending=False)
owners = ysp.percent(owners, div_round=3, to_percent=False,
                calc_columns=owners.columns, 
                percent_columns=[f'{col} %' for col in owners.columns])
owners

/home/krtsh/MEGAsync/Programming/work/Old scripts/skoda/processing.py:55: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(data[calc_columns] \


,Owner,Not defined,Potential buyer,Total,Owner %,Not defined %,Potential buyer %
OCTAVIA,1134,2379,175,3688,0.307,0.645,0.047
RAPID,699,1459,132,2290,0.305,0.637,0.058
KODIAQ,474,1192,113,1779,0.266,0.670,0.064
SUPERB,151,447,36,634,0.238,0.705,0.057
KAROQ,92,232,32,356,0.258,0.652,0.090
Total types,2550,5709,488,8747,0.292,0.653,0.056


# Тональность

In [10]:
def del_tag_num(name):
    return re.sub('\d{2}_', '', name)

tonality = pd.DataFrame()
for m in md:
    temp_df = ysp.tonality(df, [m], m)
    tonality = tonality.append(temp_df) \
                .sort_values('Total', ascending=False)
tonality = ysp.percent(tonality, div_round=3, to_percent=False, 
                calc_columns=tonality.columns, 
                percent_columns=[f'{col} %' for col in tonality.columns],) \
                .rename(index=del_tag_num)
tonality

/tmp/ipykernel_45135/3215046199.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tonality = tonality.append(temp_df) \
/tmp/ipykernel_45135/3215046199.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tonality = tonality.append(temp_df) \
/tmp/ipykernel_45135/3215046199.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tonality = tonality.append(temp_df) \
/tmp/ipykernel_45135/3215046199.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tonality = tonality.append(temp_df) \
/tmp/ipykernel_45135/3215046199.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.c

,Negative,Neutral,Positive,Total,Negative %,Neutral %,Positive %
OCTAVIA,655,2428,605,3688,0.178,0.658,0.164
RAPID,415,1433,442,2290,0.181,0.626,0.193
KODIAQ,285,1096,396,1777,0.160,0.617,0.223
SUPERB,96,415,123,634,0.151,0.655,0.194
KAROQ,39,242,74,355,0.110,0.682,0.208
Total types,1490,5614,1640,8744,0.170,0.642,0.188


# Число постов

In [11]:
others = [col for col in df.columns if col.startswith('50_') or col.startswith('44_')]

temp_dict = {'OTHERS': np.nan, 'TOTAL': np.nan}
others_count = 0

for m in others:
    others_count += df[(df[m] == m)]['URL'].count()
for m in md:
    temp_dict[re.sub('\d{2}_','',m)] =  df[(df[m] == m)]['URL'].count()

post_count  = pd.DataFrame(temp_dict, index=['Posts']) \
                .transpose() \
                .sort_values('Posts', ascending=False)
post_count.loc['TOTAL'] = post_count.sum() + others_count
post_count.loc['OTHERS'] = others_count
post_count['% from total'] = ((post_count.Posts / post_count.Posts.iloc[-1])).round(3)
post_count['% from top 5'] = (post_count.Posts / sum(post_count.Posts.iloc[:5])).round(3)
post_count = post_count

post_count

,Posts,% from total,% from top 5
OCTAVIA,3688.0,0.399,0.422
RAPID,2290.0,0.248,0.262
KODIAQ,1777.0,0.192,0.203
SUPERB,634.0,0.069,0.073
KAROQ,355.0,0.038,0.041
OTHERS,499.0,0.054,0.057
TOTAL,9243.0,1.000,1.057


# Охват уникальный

In [12]:
temp_dict = {'TOTAL': np.nan}
for m in md:
    temp_dict[re.sub('\d{2}_','',m)] = df[(df[m] == m)] \
                                        .groupby('Профиль') \
                                        .agg({'Подписчики': 'mean'}) \
                                        .sum()[0]
unique_coverage = pd.DataFrame(temp_dict,index=['Coverage']) \
                    .transpose() \
                    .sort_values('Coverage', ascending=False)
unique_coverage.loc['TOTAL'] = df.groupby('Профиль')['Подписчики'] \
                                .mean().sum()
unique_coverage

,Coverage
OCTAVIA,3070438.0
SUPERB,1143611.0
KODIAQ,1092090.0
RAPID,811848.0
KAROQ,376243.0
TOTAL,14069159.0


# ALL Слайд 7

In [13]:
nsi = ysp.nsi(df)
nsi

/home/krtsh/MEGAsync/Programming/work/Old scripts/skoda/processing.py:98: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nsi = nsi.append(nsi.sum().to_frame() \


,Негативная,Нейтральная,Позитивная,Итого,NSI
01.02.2022,71,219,104,394,0.64
02.02.2022,46,241,78,365,0.75
03.02.2022,59,284,93,436,0.73
04.02.2022,74,259,95,428,0.65
05.02.2022,55,186,81,322,0.66
06.02.2022,77,202,65,344,0.55
07.02.2022,83,182,56,321,0.48
08.02.2022,63,244,56,363,0.65
09.02.2022,70,269,77,416,0.66
10.02.2022,75,312,101,488,0.69


# Слайд 6

In [14]:
slide_6 = nsi.loc['Итого'].to_frame()[:-1]
slide_6['in %'] = (slide_6['Итого'] / slide_6['Итого'].iloc[-1]).round(3)
slide_6.rename(columns={'Итого':'Кол-во'},inplace=True)

slide_6

,Кол-во,in %
Негативная,1775.0,0.179
Нейтральная,6135.0,0.617
Позитивная,2027.0,0.204
Итого,9937.0,1.000


# Слайд 5

In [15]:
slide5_1 = df.groupby('Источник') \
             .agg({'Источник': 'count'}) \
             .rename(columns={'Источник': 'scount'}) \
             .sort_values('scount', ascending=False)[:10]
slide5_1 = slide5_1 \
        .append(pd.DataFrame([df['Источник'].nunique()], 
                             columns=['scount'])) \
        .rename({0: 'Unique source'})
slide5_1

/tmp/ipykernel_45135/823818126.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  .append(pd.DataFrame([df['Источник'].nunique()],


,scount
vk.com,2716
youtube.com,2123
telegram.me,1680
drive2.ru,1567
maps.yandex.ru,517
maps.google.com,222
instagram.com,127
facebook.com,108
tiktok.com,95
nn.ru,90


In [22]:
slide5_2 = df.groupby('Тип источника') \
             .agg({'Тип источника': 'count'}) \
             .rename(columns={'Тип источника':'count'})
slide5_2.at['Блог и форумы'] = slide5_2.loc['Блог'] + slide5_2.loc['Форум']
slide5_2 = slide5_2.iloc[np.r_[1:4, 5:6]].sort_index().astype(int)
in_perc = ysp.joining(slide5_2)
slide5_2['in %'] = in_perc

slide5_2

,count,in %
Тип источника,,
Блог и форумы,2154,2154 (22%)
Мессенджеры,1619,1619 (16%)
Отзывы,918,918 (9%)
Соц. сеть,5246,5246 (53%)


# Слайд 3

In [23]:
curm_posts = df['Дата'].count()
curm_actions = df[['Сумма всех реакций', 'Комментарии', 'Репосты']].sum().sum()
curm_ots = df.groupby('Профиль')['Подписчики'] \
             .mean().sum()

lastm_posts = int(sht.get_worksheet(0).acell('L31').value)
lastm_actions = int(sht.get_worksheet(0).acell('M31').value)
lastm_ots = int(sht.get_worksheet(0).acell('N31').value)

In [24]:
slide_3 = {'Текущий месяц': {
                'ПОСТЫ': curm_posts, 
                'ВОВЛЕЧЕНИЕ': curm_actions, 
                'ОХВАТ': curm_ots},
           'Change': {
               'ПОСТЫ': curm_posts - lastm_posts,         
               'ВОВЛЕЧЕНИЕ': curm_actions - lastm_actions, 
                'ОХВАТ': curm_ots - lastm_ots},
           'Change %': {
                'ПОСТЫ': ((curm_posts - lastm_posts) / lastm_posts).round(3), 
                'ВОВЛЕЧЕНИЕ': ((curm_actions - lastm_actions) / lastm_actions).round(3), 
                'ОХВАТ': ((curm_ots - lastm_ots) / lastm_ots).round(3)}          
          }

slide_3 = pd.DataFrame(slide_3).transpose()

slide_3

,ПОСТЫ,ВОВЛЕЧЕНИЕ,ОХВАТ
Текущий месяц,9937.00,48232.000,1.406916e+07
Change,-1896.00,-88461.000,-1.155557e+07
Change %,-0.16,-0.647,-4.510000e-01


# По моделям

In [25]:
lastm = {'40_RAPID': {
            'model_lastm_posts': int(sht.get_worksheet(0).acell('AD3').value), 
            'model_lastm_actions': int(sht.get_worksheet(0).acell('AE3').value), 
            'model_lastm_ots': int(sht.get_worksheet(0).acell('AF3').value)},
         '41_OCTAVIA': {
             'model_lastm_posts': int(sht.get_worksheet(0).acell('AD17').value), 
             'model_lastm_actions': int(sht.get_worksheet(0).acell('AE17').value), 
             'model_lastm_ots': int(sht.get_worksheet(0).acell('AF17').value)},
         '42_KODIAQ': {
             'model_lastm_posts': int(sht.get_worksheet(0).acell('AD31').value), 
             'model_lastm_actions': int(sht.get_worksheet(0).acell('AE31').value), 
             'model_lastm_ots': int(sht.get_worksheet(0).acell('AF31').value)},
         '43_SUPERB': {
             'model_lastm_posts': int(sht.get_worksheet(0).acell('AD45').value), 
             'model_lastm_actions': int(sht.get_worksheet(0).acell('AE45').value), 
             'model_lastm_ots': int(sht.get_worksheet(0).acell('AF45').value)},
         '45_KAROQ': {
             'model_lastm_posts': int(sht.get_worksheet(0).acell('AD59').value), 
             'model_lastm_actions': int(sht.get_worksheet(0).acell('AE59').value), 
             'model_lastm_ots': int(sht.get_worksheet(0).acell('AF59').value)}
        }

In [26]:
model_dynamics = {}

for m in md:
    model_curm_posts = df[(df[m] == m)]['Дата'].count()
    model_curm_actions = df[(df[m] == m)][['Сумма всех реакций', 'Комментарии', 'Репосты']].sum().sum()
    model_curm_ots = df[(df[m] == m)].groupby('Профиль')['Подписчики'].mean().sum()

    temp = {'Текущий месяц': {
                'ПОСТЫ': model_curm_posts, 
                'ВОВЛЕЧЕНИЕ': model_curm_actions, 
                'ОХВАТ': model_curm_ots},
            'Change': {
                'ПОСТЫ': model_curm_posts - lastm[m]['model_lastm_posts'], 
                'ВОВЛЕЧЕНИЕ': model_curm_actions - lastm[m]['model_lastm_actions'], 
                'ОХВАТ': model_curm_ots - lastm[m]['model_lastm_ots']},
            'Change %': {
                'ПОСТЫ': ((model_curm_posts - lastm[m]['model_lastm_posts']) / lastm[m]['model_lastm_posts']).round(3), 
                'ВОВЛЕЧЕНИЕ': ((model_curm_actions - lastm[m]['model_lastm_actions']) / lastm[m]['model_lastm_actions']).round(3), 
                'ОХВАТ': ((model_curm_ots - lastm[m]['model_lastm_ots']) / lastm[m]['model_lastm_ots']).round(3)}          
              }

    temp = pd.DataFrame(temp).transpose()
    model_dynamics['model_dynamics_' + m] = temp

model_dynamics

{'model_dynamics_40_RAPID':                   ПОСТЫ  ВОВЛЕЧЕНИЕ        ОХВАТ
 Текущий месяц  2290.000    7891.000   811848.000
 Change         -527.000   -3096.000 -1790391.000
 Change %         -0.187      -0.282       -0.688,
 'model_dynamics_41_OCTAVIA':                   ПОСТЫ  ВОВЛЕЧЕНИЕ        ОХВАТ
 Текущий месяц  3688.000   18641.000  3070438.000
 Change         -791.000  -20087.000 -3253861.000
 Change %         -0.177      -0.519       -0.515,
 'model_dynamics_42_KODIAQ':                   ПОСТЫ  ВОВЛЕЧЕНИЕ        ОХВАТ
 Текущий месяц  1777.000    9047.000  1092090.000
 Change         -540.000     257.000   575330.000
 Change %         -0.233       0.029        1.113,
 'model_dynamics_43_SUPERB':                  ПОСТЫ  ВОВЛЕЧЕНИЕ        ОХВАТ
 Текущий месяц  634.000    3935.000  1143611.000
 Change         -19.000    -792.000 -2538788.000
 Change %        -0.029      -0.168       -0.689,
 'model_dynamics_45_KAROQ':                  ПОСТЫ  ВОВЛЕЧЕНИЕ       ОХВАТ
 Текущий меся

# По моделям

In [27]:
model_tonality = {}
for m in md:
    temp = {m: 
            {'Негативная': tonality['Negative %'].loc[re.sub('\d{2}_','',m)], 
            'Нейтральная': tonality['Neutral %'].loc[re.sub('\d{2}_','',m)], 
            'Позитивная': tonality['Positive %'].loc[re.sub('\d{2}_','',m)]}}

    temp = pd.DataFrame(temp)
    model_tonality['model_tonality_' + m] = temp

model_tonality['model_tonality_40_RAPID']

,40_RAPID
Негативная,0.181
Нейтральная,0.626
Позитивная,0.193


In [28]:
model_source_dict={}
for m in md:
    temp = df[(df[m] == m)].groupby('Источник') \
                           .agg({'Источник': 'count'}) \
                           .rename(columns={'Источник': m}) \
                           .sort_values(m, ascending=False)[:10]
    temp = temp \
            .append(pd.DataFrame([df[(df[m] == m)]['Источник'].nunique()], columns=[m])) \
            .rename({0: 'Unique source'})

    model_source_dict['model_sources_' + m] = temp
model_source_dict['model_sources_40_RAPID']

/tmp/ipykernel_45135/2920118681.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  .append(pd.DataFrame([df[(df[m] == m)]['Источник'].nunique()], columns=[m])) \
/tmp/ipykernel_45135/2920118681.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  .append(pd.DataFrame([df[(df[m] == m)]['Источник'].nunique()], columns=[m])) \
/tmp/ipykernel_45135/2920118681.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  .append(pd.DataFrame([df[(df[m] == m)]['Источник'].nunique()], columns=[m])) \
/tmp/ipykernel_45135/2920118681.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  .append(pd.DataFrame([df[(df[m] == m)]['Источник'].nunique()], columns=

,40_RAPID
vk.com,758
drive2.ru,448
telegram.me,440
youtube.com,368
maps.yandex.ru,49
maps.google.com,31
forums.drom.ru,17
yell.ru,16
tiktok.com,13
pikabu.ru,13


In [29]:
model_source_type = {}
for m in md:    
    temp = df[(df[m] == m)].groupby('Тип источника')['Тип источника'] \
                           .count().to_frame() \
                           .rename(columns={'Тип источника':'count'})
    # temp = df[(df[m] == m)].groupby('Тип источника') \
    #                        .agg({'Тип источника': 'count'}) \
    #                        .rename(columns={'Тип источника':'count'}
    temp.at['Блог и форумы'] = temp.loc['Блог'] + temp.loc['Форум']
    # print(temp)
    temp = temp.sort_index().astype(int)
    temp = temp.iloc[1:6]
    in_perc = ysp.joining(temp)
    temp['in %'] = in_perc
    model_source_type['model_source_type_' + m] = temp    
model_source_type['model_source_type_40_RAPID']

,count,in %
Тип источника,,
Блог и форумы,559,559 (20%)
Мессенджеры,430,430 (15%)
Отзывы,131,131 (5%)
Соц. сеть,1170,1170 (42%)
Форум,524,524 (19%)


In [30]:
model_city = {}
for m in md:
    temp = df[(df[m] == m)].groupby('Город') \
                           .agg({'Город': 'count'}) \
                           .rename(columns={'Город': m}) \
                           .sort_values(m, ascending=False)[:10]
    model_city['model_city_' + m] = temp.astype(int)
model_city['model_city_40_RAPID']

,40_RAPID
Город,
Москва,125
Санкт-Петербург,90
Барнаул,21
Рязань,20
Сергиев Посад,20
Екатеринбург,19
Смоленск,17
Самара,16
Казань,15


In [31]:
model_comparison = {}
for m in md:
    temp = df_count[m].sort_values(ascending=False)[:5].to_frame()
    temp = temp.append(pd.DataFrame({m: df_count[m].sum()}, index=['Total']))
    temp['in %'] = (temp / temp.loc['Total']).round(3)
    model_comparison['model_comparison_' + m] = temp   
model_comparison['model_comparison_40_RAPID']

/tmp/ipykernel_45135/3226905154.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.DataFrame({m: df_count[m].sum()}, index=['Total']))
/tmp/ipykernel_45135/3226905154.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.DataFrame({m: df_count[m].sum()}, index=['Total']))
/tmp/ipykernel_45135/3226905154.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.DataFrame({m: df_count[m].sum()}, index=['Total']))
/tmp/ipykernel_45135/3226905154.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.DataFrame({m: df_count[m].sum()}, index=['Total']))
/tmp/ipykern

,40_RAPID,in %
80_Volkswagen,272,0.318
80_ВАЗ (Lada),161,0.188
80_Hyundai,120,0.140
80_Kia,110,0.129
80_Renault,45,0.053
Total,856,1.000


In [20]:
all = pd.DataFrame({'Негативная': [],'Нейтральная': [],'Позитивная': [],'Всего': []})

for col in df.columns:
    if str(col).startswith(feature_tags):
        neg = df[df['Тональность'] == 'Негативная'][col].count()
        neu = df[df['Тональность'] == 'Нейтральная'][col].count()
        pos = df[df['Тональность'] == 'Позитивная'][col].count()
        temp = pd.DataFrame({col: {'Негативная': neg, 
                                    'Нейтральная': neu, 
                                    'Позитивная': pos, 
                                    'Всего': sum([neg, neu, pos])}}).transpose()
        all = all.append(temp).sort_values('Всего', ascending=False)
all = all.rename(index={
                                        '14_Динамика': 'Dynamics',
                                        '17_Поломка': 'Breakdowns',
                                        '03_Дизайн': 'Design',
                                        '01_Цена': 'Price',
                                        '12_Качество': 'Quality',
                                        '15_Комплектация': 'Package',
                                        '22_Расход/обслуживание': 'Car care',
                                        '09_Технологии': 'Technologies',
                                        '11_Надежность': 'Reliability',
                                        '06_Комфорт': 'Comfort',
                                        '20_Расход топлива': 'Fuel consumption',
                                        '18_Такси': 'Taxi',
                                        '08_Багажник': 'Trunk',
                                        '04_Салон': 'Car interior',
                                        '13_Проходимость': 'Vehicle flotation',
                                        '07_Вместительность': 'Capacity',
                                        '02_ Ценность': 'Value',
                                        '05_Семья': 'Family',
                                        '10_Кастомизация': 'Customization',
                                        '16_Ржавчина': 'Rust',
                                        '21_Отзыв': 'Review',
                                        '19_Simply Clever': 'Simply Clever'})
print(all)

/tmp/ipykernel_9526/1021771712.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all = all.append(temp).sort_values('Всего', ascending=False)
/tmp/ipykernel_9526/1021771712.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all = all.append(temp).sort_values('Всего', ascending=False)
/tmp/ipykernel_9526/1021771712.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all = all.append(temp).sort_values('Всего', ascending=False)
/tmp/ipykernel_9526/1021771712.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all = all.append(temp).sort_values('Всего', ascending=False)
/tmp/ipykernel_9526/1021771712.py:12: FutureWarning: The frame.a

                   Негативная  Нейтральная  Позитивная  Всего
Price                    51.0        126.0        24.0  201.0
Dynamics                  8.0        115.0        33.0  156.0
Quality                  31.0         94.0        23.0  148.0
Package                   9.0         79.0        26.0  114.0
Design                    9.0         78.0        12.0   99.0
Technologies              6.0         57.0         9.0   72.0
Comfort                   8.0         43.0        15.0   66.0
Vehicle flotation         2.0         27.0        12.0   41.0
Car care                  6.0         15.0         8.0   29.0
Breakdowns                8.0         18.0         2.0   28.0
Car interior              3.0         21.0         3.0   27.0
Fuel consumption          1.0         15.0         6.0   22.0
Taxi                      1.0          9.0         0.0   10.0
Reliability               1.0          3.0         5.0    9.0
Customization             2.0          6.0         0.0    8.0
Capacity

/tmp/ipykernel_9526/1021771712.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all = all.append(temp).sort_values('Всего', ascending=False)
/tmp/ipykernel_9526/1021771712.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all = all.append(temp).sort_values('Всего', ascending=False)


In [32]:
models_ft = {}
for m in md:    
    models_ft['models_ft_' + m] = pd.DataFrame({'Негативная': [],'Нейтральная': [],'Позитивная': [],'Всего': []})
#     ft = pd.DataFrame({'Негативная': [],'Нейтральная': [],'Позитивная': [],'Всего': []})
    for col in df.columns:
            if str(col).startswith(feature_tags):
                neg = df[(df[m] == m) & (df['Тональность'] == 'Негативная')][col].count()
                neu = df[(df[m] == m) & (df['Тональность'] == 'Нейтральная')][col].count()
                pos = df[(df[m] == m) & (df['Тональность'] == 'Позитивная')][col].count()
                temp = pd.DataFrame({col: {'Негативная': neg, 
                                           'Нейтральная': neu, 
                                           'Позитивная': pos, 
                                           'Всего': sum([neg, neu, pos])}}).transpose()
                
                models_ft['models_ft_' + m] = models_ft['models_ft_' + m].append(temp) \
                                                .sort_values('Всего', ascending=False)

                models_ft['models_ft_' + m] = models_ft['models_ft_' + m] \
                                    .rename(index={
                                        '14_Динамика': 'Dynamics',
                                        '17_Поломка': 'Breakdowns',
                                        '03_Дизайн': 'Design',
                                        '01_Цена': 'Price',
                                        '12_Качество': 'Quality',
                                        '15_Комплектация': 'Package',
                                        '22_Расход/обслуживание': 'Car care',
                                        '09_Технологии': 'Technologies',
                                        '11_Надежность': 'Reliability',
                                        '06_Комфорт': 'Comfort',
                                        '20_Расход топлива': 'Fuel consumption',
                                        '18_Такси': 'Taxi',
                                        '08_Багажник': 'Trunk',
                                        '04_Салон': 'Car interior',
                                        '13_Проходимость': 'Vehicle flotation',
                                        '07_Вместительность': 'Capacity',
                                        '02_ Ценность': 'Value',
                                        '05_Семья': 'Family',
                                        '10_Кастомизация': 'Customization',
                                        '16_Ржавчина': 'Rust',
                                        '21_Отзыв': 'Review',
                                        '19_Simply Clever': 'Simply Clever'})

models_ft

/tmp/ipykernel_45135/2047177685.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  models_ft['models_ft_' + m] = models_ft['models_ft_' + m].append(temp) \
/tmp/ipykernel_45135/2047177685.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  models_ft['models_ft_' + m] = models_ft['models_ft_' + m].append(temp) \
/tmp/ipykernel_45135/2047177685.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  models_ft['models_ft_' + m] = models_ft['models_ft_' + m].append(temp) \
/tmp/ipykernel_45135/2047177685.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  models_ft['models_ft_' + m] = models_ft['models_ft_' + m].append(temp) \
/tmp/ipykern

{'models_ft_40_RAPID':                    Негативная  Нейтральная  Позитивная  Всего
 Package                  91.0        371.0       111.0  573.0
 Price                   134.0        297.0        81.0  512.0
 Quality                  98.0        157.0        72.0  327.0
 Dynamics                 34.0        211.0        81.0  326.0
 Technologies             51.0        198.0        35.0  284.0
 Design                   32.0        157.0        41.0  230.0
 Comfort                  54.0        127.0        39.0  220.0
 Car care                 17.0         95.0        25.0  137.0
 Car interior             28.0         70.0        31.0  129.0
 Vehicle flotation        19.0         59.0        18.0   96.0
 Taxi                     12.0         61.0         7.0   80.0
 Breakdowns               30.0         46.0         2.0   78.0
 Value                    20.0         35.0        10.0   65.0
 Fuel consumption          5.0         34.0        16.0   55.0
 Trunk                     6.0   

In [33]:
# Для выводов на 6м слайде
models_tags = pd.DataFrame({'Негативная': [],'Нейтральная': [],'Позитивная': [],'Всего': []})
for col in df.columns:
    if str(col).startswith(feature_tags):
        neg = df.query('Тональность == "Негативная"')[col].count()
        neu = df.query('Тональность == "Нейтральная"')[col].count()
        pos = df.query('Тональность == "Позитивная"')[col].count()
        temp = pd.DataFrame({col: {'Негативная': neg, 
                                   'Нейтральная': neu, 
                                   'Позитивная': pos, 
                                   'Всего': sum([neg, neu, pos])}}).transpose()
        models_tags = models_tags.append(temp) \
                               .sort_values('Всего', ascending=False)
models_tags

/tmp/ipykernel_45135/2161917578.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  models_tags = models_tags.append(temp) \
/tmp/ipykernel_45135/2161917578.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  models_tags = models_tags.append(temp) \
/tmp/ipykernel_45135/2161917578.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  models_tags = models_tags.append(temp) \
/tmp/ipykernel_45135/2161917578.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  models_tags = models_tags.append(temp) \
/tmp/ipykernel_45135/2161917578.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future vers

,Негативная,Нейтральная,Позитивная,Всего
15_Комплектация,426.0,1507.0,417.0,2350.0
01_Цена,477.0,1078.0,275.0,1830.0
14_Динамика,180.0,980.0,318.0,1478.0
12_Качество,332.0,621.0,244.0,1197.0
09_Технологии,177.0,711.0,115.0,1003.0
03_Дизайн,107.0,595.0,177.0,879.0
06_Комфорт,152.0,442.0,136.0,730.0
22_Расход/обслуживание,87.0,380.0,115.0,582.0
13_Проходимость,69.0,338.0,116.0,523.0
04_Салон,98.0,292.0,98.0,488.0


# ВЫГРУЗКА

In [21]:
c = gspread_pandas.conf.get_config(conf_dir='./', file_name='./google_secret.json')
spread = Spread('Skoda SL Report', config=c)
spread.sheets

[<Worksheet 'data' id:236867346>,
 <Worksheet 'data (копия)' id:1020131371>,
 <Worksheet 'charts' id:0>]

In [29]:
# ПОПРАВЬ РЕГИОН
spread.df_to_sheet(all, sheet='data', start='AO2')

In [30]:
spread.df_to_sheet(post_count, sheet='data', start='A2')
spread.df_to_sheet(tonality, sheet='data', start='F2') 
spread.df_to_sheet(owners, sheet='data', start='F11') 
spread.df_to_sheet(recommend, sheet='data', start='F20')

spread.df_to_sheet(nsi, sheet='data', start='A30') 
spread.df_to_sheet(slide5_1, sheet='data', start='H30') 
spread.df_to_sheet(slide5_2, sheet='data', start='H44') 
spread.df_to_sheet(slide_6, sheet='data', start='H53') 
spread.df_to_sheet(slide_3, sheet='data', start='K30') 

spread.df_to_sheet(model_source_dict['model_sources_40_RAPID'], sheet='data', start='R2') 
spread.df_to_sheet(model_source_dict['model_sources_41_OCTAVIA'], sheet='data', start='R16') 
spread.df_to_sheet(model_source_dict['model_sources_42_KODIAQ'], sheet='data', start='R30') 
spread.df_to_sheet(model_source_dict['model_sources_43_SUPERB'], sheet='data', start='R44') 
spread.df_to_sheet(model_source_dict['model_sources_45_KAROQ'], sheet='data', start='R58')

spread.df_to_sheet(model_city['model_city_40_RAPID'], sheet='data', start='V2') 
spread.df_to_sheet(model_city['model_city_41_OCTAVIA'], sheet='data', start='V16') 
spread.df_to_sheet(model_city['model_city_42_KODIAQ'], sheet='data', start='V30') 
spread.df_to_sheet(model_city['model_city_43_SUPERB'], sheet='data', start='V44') 
spread.df_to_sheet(model_city['model_city_45_KAROQ'], sheet='data', start='V58') 

spread.df_to_sheet(model_source_type['model_source_type_40_RAPID'], sheet='data', start='Y2') 
spread.df_to_sheet(model_source_type['model_source_type_41_OCTAVIA'], sheet='data', start='Y16') 
spread.df_to_sheet(model_source_type['model_source_type_42_KODIAQ'], sheet='data', start='Y30') 
spread.df_to_sheet(model_source_type['model_source_type_43_SUPERB'], sheet='data', start='Y44') 
spread.df_to_sheet(model_source_type['model_source_type_45_KAROQ'], sheet='data', start='Y58') 

spread.df_to_sheet(model_dynamics['model_dynamics_40_RAPID'], sheet='data', start='AC2') 
spread.df_to_sheet(model_dynamics['model_dynamics_41_OCTAVIA'], sheet='data', start='AC16') 
spread.df_to_sheet(model_dynamics['model_dynamics_42_KODIAQ'], sheet='data', start='AC30') 
spread.df_to_sheet(model_dynamics['model_dynamics_43_SUPERB'], sheet='data', start='AC44') 
spread.df_to_sheet(model_dynamics['model_dynamics_45_KAROQ'], sheet='data', start='AC58') 

spread.df_to_sheet(model_comparison['model_comparison_40_RAPID'], sheet='data', start='AH2') 
spread.df_to_sheet(model_comparison['model_comparison_41_OCTAVIA'], sheet='data', start='AH16') 
spread.df_to_sheet(model_comparison['model_comparison_42_KODIAQ'], sheet='data', start='AH30') 
spread.df_to_sheet(model_comparison['model_comparison_43_SUPERB'], sheet='data', start='AH44') 
spread.df_to_sheet(model_comparison['model_comparison_45_KAROQ'], sheet='data', start='AH58') 

spread.df_to_sheet(model_tonality['model_tonality_40_RAPID'], sheet='data', start='AL2') 
spread.df_to_sheet(model_tonality['model_tonality_41_OCTAVIA'], sheet='data', start='AL16') 
spread.df_to_sheet(model_tonality['model_tonality_42_KODIAQ'], sheet='data', start='AL30') 
spread.df_to_sheet(model_tonality['model_tonality_43_SUPERB'], sheet='data', start='AL44') 
spread.df_to_sheet(model_tonality['model_tonality_45_KAROQ'], sheet='data', start='AL58') 

spread.df_to_sheet(models_ft['models_ft_40_RAPID'], sheet='data', start='AO2') 
spread.df_to_sheet(models_ft['models_ft_41_OCTAVIA'], sheet='data', start='AO27') 
spread.df_to_sheet(models_ft['models_ft_42_KODIAQ'], sheet='data', start='AO52') 
spread.df_to_sheet(models_ft['models_ft_43_SUPERB'], sheet='data', start='AO77') 
spread.df_to_sheet(models_ft['models_ft_45_KAROQ'], sheet='data', start='AO102') 



# spread.df_to_sheet(, sheet='data', start='') 